In [ ]:
import pandas as pd
import random
import datetime
#Importamos los ficheros de reservas y hoteles:
bookings = pd.read_excel('bookings.xlsx')
hotels = pd.read_excel('hotels.xlsx')
#Se verifica la no existencia de valores null en los ficheros:
print('Fichero bookings.xlsx:')
print(bookings.isnull().sum(), '\n')
print('Fichero hotels.xlsx:')
print(hotels.isnull().sum(), '\n')
#Generamos los listados de segmentos, clientes y regímenes que asignaremos:
listSegments = ['Hoteles sostenibles', 'Solo adultos', 'Hoteles que aceptan mascotas', 'LGTBI friendly', 'Hoteles de negocios', 'Hoteles rurales', 'Hoteles para familias', 'Hoteles con actividades deportivas', 'Sin segmento']
listClients = ['Travel fun', 'Viajes Fernández', 'A todo viaje', 'Travel with us', 'Viajes paraiso', 'Viaja sin límites', 'Party travel', 'Unique experiences travel', 'World travel']
listBoards = ['Solo alojamiento', 'Alojamiento y desayuno', 'Todo incluido', 'Media pensión', 'Pensión completa']
bookings['Hotel ID'] = None
bookings['Roomnights'] = None
bookings['Total Price'] = None
bookings['Client'] = None
bookings['Lead time bucket'] = None
bookings['Arrival date'] = None
bookings['Booking date'] = None
format = '%d/%m/%Y'
hotels['Segment'] = None
#Asignamos el segmento del hotel:
for h in range(len(hotels)):
  hotels['Segment'][h] = random.choices(listSegments, weights = [3,4,2,3,2,2,2,1,5])
  hotels['Segment'][h] = str(hotels['Segment'][h]).replace("'", "")
  hotels['Segment'][h] = str(hotels['Segment'][h]).replace("]", "")
  hotels['Segment'][h] = str(hotels['Segment'][h]).replace("[", "")
#Recorremos cada una de las reservas:
for b in range(len(bookings)):
  #Eliminamos las líneas de reservas en las que no hay adultos o más de 3 niños
  if bookings['no_of_adults'][b] == 0 or bookings['no_of_children'][b] > 3 or (bookings['no_of_weekend_nights'][b] + bookings['no_of_week_nights'][b] == 0):
    bookings = bookings.drop(b)
    continue
  #Asignamos un hotel, si hay niños se descartan los segmentos de solo adultos y hoteles de negocios y si no los hay se descarta el segmento de hoteles para familias:
  if (bookings['no_of_children'][b]) > 1:
    hotelsAux = hotels[(hotels.Segment.isin(["Hoteles sostenibles", "Hoteles para familias", "Hoteles que aceptan mascotas", "LGTBI friendly", "Hoteles rurales", "Hoteles con actividades deportivas", "Sin segmento"]))]
  else:
    hotelsAux = hotels[(hotels.Segment.isin(["Hoteles sostenibles", "Solo adultos", "Hoteles que aceptan mascotas", "LGTBI friendly", "Hoteles de negocios", "Hoteles rurales", "Hoteles con actividades deportivas", "Sin segmento"]))]
  hotels_id = list(hotelsAux['Hotel ID'])
  bookings['Hotel ID'][b] = random.choice(hotels_id)
  #Asignamos un cliente:
  bookings['Client'][b] = random.choices(listClients, weights = [7, 5, 1, 2, 1, 3, 1, 2, 1])
  bookings['Client'][b] = str(bookings['Client'][b]).replace("'", "")
  bookings['Client'][b] = str(bookings['Client'][b]).replace("[", "")
  bookings['Client'][b] = str(bookings['Client'][b]).replace("]", "")
  #Asignamos un régimen:
  bookings['type_of_meal_plan'][b] = random.choices(listBoards, weights = [10, 5, 1, 2, 1])
  bookings['type_of_meal_plan'][b] = str(bookings['type_of_meal_plan'][b]).replace("'", "")
  bookings['type_of_meal_plan'][b] = str(bookings['type_of_meal_plan'][b]).replace("]", "")
  bookings['type_of_meal_plan'][b] = str(bookings['type_of_meal_plan'][b]).replace("[", "")
  #Obtenemos el número total de noches:
  bookings['Roomnights'][b] = bookings['no_of_weekend_nights'][b] + bookings['no_of_week_nights'][b]
  #Obtenemos el precio total de la reserva:
  bookings['Total Price'][b] = bookings['Roomnights'][b] * bookings['avg_price_per_room'][b]
  #Obtenemos la fecha de llegada:
  fecha = str(bookings['arrival_date'][b]) + '/' + str(bookings['arrival_month'][b]) + '/' + str(bookings['arrival_year'][b])
  bookings['Arrival date'][b] = datetime.datetime.strptime(fecha, format)
  bookings['Arrival date'][b] = bookings['Arrival date'][b].date()
  #Obtenemos la fecha de reserva:
  bookings['Booking date'][b] = bookings['Arrival date'][b] - datetime.timedelta(days=int(bookings['lead_time'][b]))
  #Asignamos los rangos de antelación de la reserva:
  if bookings['lead_time'][b] == 0 or bookings['lead_time'][b] == 1:
    bookings['Lead time bucket'][b] = 'De 0 a 1 día'
  elif bookings['lead_time'][b] > 1 and bookings['lead_time'][b] <= 7:
    bookings['Lead time bucket'][b] = 'De 2 a 7 días'
  elif bookings['lead_time'][b] > 7 and bookings['lead_time'][b] <= 14:
    bookings['Lead time bucket'][b] = 'De 8 a 14 días'
  elif bookings['lead_time'][b] > 15 and bookings['lead_time'][b] <= 30:
    bookings['Lead time bucket'][b] = 'De 15 a 30 días'
  elif bookings['lead_time'][b] > 31 and bookings['lead_time'][b] <= 90:
    bookings['Lead time bucket'][b] = 'De 1 a 3 meses'
  elif bookings['lead_time'][b] > 91 and bookings['lead_time'][b] <= 180:
    bookings['Lead time bucket'][b] = 'De 3 a 6 meses'
  else:
    bookings['Lead time bucket'][b] = 'Más de 6 meses'
  #Asignamos el tipo de habitación en función de la ocupación:
  if (bookings['no_of_adults'][b] + bookings['no_of_children'][b]) == 1:
    bookings['room_type_reserved'][b] = random.choices(['Individual estándar', 'Individual superior', 'Individual con balcón', 'Individual cama matrimonio'], weights = [3, 2, 1, 1])
  elif (bookings['no_of_adults'][b] + bookings['no_of_children'][b]) == 2:
    bookings['room_type_reserved'][b] = random.choices(['Doble estándar', 'Doble superior', 'Doble con balcón', 'Doble cama matrimonio', 'Doble 2 camas'], weights = [3, 2, 1, 2, 1])
  elif (bookings['no_of_adults'][b] + bookings['no_of_children'][b]) == 3:
    bookings['room_type_reserved'][b] = random.choices(['Triple estándar', 'Triple superior', 'Triple con balcón', 'Triple 2 dormitorios', 'Triple 3 camas'], weights = [3, 2, 1, 2, 1])
  else:
    bookings['room_type_reserved'][b] = random.choices(['Cuádruple estándar', 'Apartamento 3 habitaciones', 'Cuádruple 3 camas'], weights = [3, 2, 1])
  bookings['room_type_reserved'][b] = str(bookings['room_type_reserved'][b]).replace("'", "")
  bookings['room_type_reserved'][b] = str(bookings['room_type_reserved'][b]).replace("]", "")
  bookings['room_type_reserved'][b] = str(bookings['room_type_reserved'][b]).replace("[", "")
#Generamos los ficheros con las transformaciones:
bookings.to_excel('bookings_processed.xlsx', index=False)
hotels.to_excel('hotels_processed.xlsx', index=False)